[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nils-holmberg/scom-org/blob/main/jnb/socm_sa.ipynb)

# social media, nlp, text analysis

## load text data

In [1]:
import pandas as pd

# load fb post dataset
fp = "../csv/fb-sa-230314.csv"
fp = "https://raw.githubusercontent.com/nils-holmberg/scom-org/main/csv/fb-sa-230314.csv"
#dtp = read.table(fn, sep='\t', header=T, strip.white=TRUE, stringsAsFactors=FALSE)
#dtp = dtp |> mutate(month=month-30)
df = pd.read_csv(fp, header=0, sep="\t")
df.head()

,id,text,org,time,lang,follow,e_lc,e_cc,e_sc,e_rc,...,group,date,bins,index,month,sa_int_abs,sa_int,wc,out,org_type
0,PlanSverige-10154245440447004,Hur är det att föda barn i ett land som drabba...,PlanSverige,1.446360e+09,sv,37539,525,10,21,0,...,25,2015-11-01,201511,2588,31,3,9,61,1,9
1,oxfamsverige-654404778032860,Lilla Jana 16 månader är en av miljontals männ...,oxfamsverige,1.446365e+09,sv,8838,103,0,11,0,...,25,2015-11-01,201511,2589,31,6,36,80,0,9
2,hallsverigerent-10153131904127477,Heja Smögens fiskauktion och havochvatten! Til...,hallsverigerent,1.446366e+09,sv,11288,192,6,16,0,...,25,2015-11-01,201511,2590,31,2,4,22,1,5
3,ClownerutanGranser-995657940490841,Ikväll gästar vår vän Suzanne Reuter även känd...,ClownerutanGranser,1.446367e+09,sv,7449,219,2,10,0,...,25,2015-11-01,201511,2591,31,3,9,51,0,9
4,lakareutangranser-10153338180335787,Den 8:e november är det Fars dag! Fira med en ...,lakareutangranser,1.446408e+09,sv,364399,510,1,45,0,...,25,2015-11-01,201511,2594,31,6,16,45,0,3


In [2]:
df['lang'].value_counts()

sv    90586
en     6602
Name: lang, dtype: int64

In [3]:
df.loc[995:1005,["id","text"]]

,id,text
995,SLMK.1981-1238466769502481,Vi måste lära oss av vår historia.
996,raddningsmissionen-10153378639580345,Nu är julklappsinsamlingen på Gothia Towers i ...
997,GbgStadsmission-1107919595899887,Göteborgs Stadsmissions julfilm sätter i år fo...
998,svenskalivraddningssallskapet-902883429759128,Nu börjar isarna lägga sig. Härliga dagar med ...
999,civilrightsdefenders-916451015059425,ETHIOPIA UPDATE: A few months after being rele...
1000,rodakorset-1079984975365462,Just nu diskuteras det mycket hur vi ska ta ha...
1001,Erikshjalpen-1052622464768598,Nu har vi fyllt på med nya Dece-prylar i gåvob...
1002,HRFriks-912182902170613,”Hur mår svensk funktionshinderforskning?” är ...
1003,sjoraddning-10153849707263939,Våra sjöräddare från RS Skillinge genomförde u...
1004,GbgStadsmission-1107957849229395,Det börjar bli julstämning i Göteborg... och s...


## sentence tokenize

In [4]:
import nltk
from nltk.tokenize import sent_tokenize
#nltk.download('popular')

# Tokenize the text into sentences and create a new DataFrame
sentences_df = pd.DataFrame(columns=['id', 'sentence_number', 'sentence'])

if False:
    for idx, row in df.loc[995:1005,["id","text"]].iterrows():
        text_id = row['id']
        text = row['text']
        sentences = sent_tokenize(text)

        # Create a DataFrame for sentences in the current text
        sentences_data = {
            'id': [text_id] * len(sentences),
            'sentence_number': list(range(1, len(sentences) + 1)),
            'sentence': sentences
        }

        # Concatenate the current text's sentences with the overall sentences DataFrame
        sentences_df = pd.concat([sentences_df, pd.DataFrame(sentences_data)], ignore_index=True)

    # Print the final DataFrame
    sentences_df


In [ ]:
!gdown https://drive.google.com/uc?id=1EMzJxxoBaN_NbvF7xhoc09K82vQ6H_LX

In [5]:
# Specify your desired output file path
fp = 'fb-sa-231228.tsv'
# Save or load the DataFrame as a TSV file
#sentences_df.to_csv(fp, sep='\t', index=False)
sentences_df = pd.read_csv(fp, header=0, sep="\t")
sentences_df.head()

,id,sentence_number,sentence
0,PlanSverige-10154245440447004,1,Hur är det att föda barn i ett land som drabba...
1,PlanSverige-10154245440447004,2,På Plan International har vi löst det så att v...
2,PlanSverige-10154245440447004,3,Hittills har 13 033 kvinnor fått hjälp och sup...
3,oxfamsverige-654404778032860,1,Lilla Jana 16 månader är en av miljontals männ...
4,oxfamsverige-654404778032860,2,Familjen bor nu i ett flyktingläger i grannlan...


In [7]:
sentences_df.shape

(423977, 3)

In [9]:
# Number of smaller DataFrames
num_splits = 10

# Calculate the number of rows per smaller DataFrame
rows_per_split = len(sentences_df) // num_splits

# Initialize a list to store the smaller DataFrames
smaller_dfs = []

# Split the large DataFrame into smaller DataFrames
for i in range(num_splits):
    start_row = i * rows_per_split
    end_row = (i + 1) * rows_per_split if i < num_splits - 1 else None
    smaller_df = sentences_df.iloc[start_row:end_row]
    smaller_dfs.append(smaller_df)

# Access the smaller DataFrames as needed with leading zeroes in their names
for i, smaller_df in enumerate(smaller_dfs):
    # Format the name with three leading zeroes
    fp = f"fb-sa-231228-{i + 1:03d}.tsv"
    print(fp)
    smaller_df.to_csv(fp, sep='\t', index=False)

fb-sa-231228-001.tsv
fb-sa-231228-002.tsv
fb-sa-231228-003.tsv
fb-sa-231228-004.tsv
fb-sa-231228-005.tsv
fb-sa-231228-006.tsv
fb-sa-231228-007.tsv
fb-sa-231228-008.tsv
fb-sa-231228-009.tsv
fb-sa-231228-010.tsv


In [11]:
# Specify your desired output file path
fp = 'fb-sa-231228-001.tsv'

sentences_df = pd.read_csv(fp, header=0, sep="\t")
sentences_df.shape

(42397, 3)

## sentiment analysis

In [6]:
!pip install -q transformers


### untrained model

In [ ]:
from transformers import pipeline

# Perform sentiment analysis using Hugging Face Transformers
sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-multilingual-cased")

sentiment_analyzer("i love this movie!")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'label': 'LABEL_0', 'score': 0.5471532344818115}]

In [ ]:
if False:
    # Iterate through each sentence and analyze sentiment
    sentences_df['sentiment'] = sentences_df['sentence'].apply(lambda x: sentiment_analyzer(x)[0])
    sentences_df.head()


### trained model

In [12]:
from transformers import pipeline

sentiment_analyzer = pipeline(
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student",
    return_all_scores=True
)

# english
sentiment_analyzer("i love this movie!")


config.json:   0%|          | 0.00/759 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'positive', 'score': 0.9912461638450623},
  {'label': 'neutral', 'score': 0.0058813379146158695},
  {'label': 'negative', 'score': 0.0028724849689751863}]]

In [13]:
# swedish
sentiment_analyzer("jag älskar den här filmen!")


[[{'label': 'positive', 'score': 0.7703008651733398},
  {'label': 'neutral', 'score': 0.061975058168172836},
  {'label': 'negative', 'score': 0.16772407293319702}]]

### cpu time

- 42K rows = 46 min (1K/min)

In [14]:
# Iterate through each sentence and analyze sentiment
sentences_df['sentiment'] = sentences_df['sentence'].apply(lambda x: sentiment_analyzer(x)[0])

sentences_df.head()

,id,sentence_number,sentence,sentiment
0,PlanSverige-10154245440447004,1,Hur är det att föda barn i ett land som drabba...,"[{'label': 'positive', 'score': 0.104590408504..."
1,PlanSverige-10154245440447004,2,På Plan International har vi löst det så att v...,"[{'label': 'positive', 'score': 0.420843392610..."
2,PlanSverige-10154245440447004,3,Hittills har 13 033 kvinnor fått hjälp och sup...,"[{'label': 'positive', 'score': 0.759050607681..."
3,oxfamsverige-654404778032860,1,Lilla Jana 16 månader är en av miljontals männ...,"[{'label': 'positive', 'score': 0.264799565076..."
4,oxfamsverige-654404778032860,2,Familjen bor nu i ett flyktingläger i grannlan...,"[{'label': 'positive', 'score': 0.467231452465..."


In [24]:
# Specify your desired output file path
fp = 'fb-sa-231228-001.tsv'
# Save or load the DataFrame as a TSV file
sentences_df.to_csv(fp, sep='\t', index=False)


In [32]:
# Function to extract the highest score and its corresponding label from a list of dictionaries
def extract_highest_score(sentiments):
    # Find the label with the highest score
    highest_score_label = max(sentiments, key=lambda x: x['score'])

    # Return the label and its score
    return highest_score_label['label'], highest_score_label['score']

# Apply the function to the dataframe
sentences_df['sa_label'], sentences_df['sa_score'] = zip(*sentences_df['sentiment'].apply(extract_highest_score))

# Display the first few rows of the updated dataframe
sentences_df.head()


,id,sentence_number,sentence,sentiment,sa_label,sa_score
0,PlanSverige-10154245440447004,1,Hur är det att föda barn i ett land som drabba...,"[{'label': 'positive', 'score': 0.104590408504...",negative,0.787572
1,PlanSverige-10154245440447004,2,På Plan International har vi löst det så att v...,"[{'label': 'positive', 'score': 0.420843392610...",negative,0.440226
2,PlanSverige-10154245440447004,3,Hittills har 13 033 kvinnor fått hjälp och sup...,"[{'label': 'positive', 'score': 0.759050607681...",positive,0.759051
3,oxfamsverige-654404778032860,1,Lilla Jana 16 månader är en av miljontals männ...,"[{'label': 'positive', 'score': 0.264799565076...",negative,0.573133
4,oxfamsverige-654404778032860,2,Familjen bor nu i ett flyktingläger i grannlan...,"[{'label': 'positive', 'score': 0.467231452465...",positive,0.467231


In [33]:
# Mapping for converting labels to numeric values
label_mapping = {
    'positive': 1,
    'neutral': 0,
    'negative': -1
}

# Function to convert labels to numeric values
def convert_label_to_numeric(label):
    return label_mapping.get(label, None)

# Create a new column with converted numeric values
sentences_df['sa_numeric'] = sentences_df['sa_label'].apply(convert_label_to_numeric)

sentences_df.head()


,id,sentence_number,sentence,sentiment,sa_label,sa_score,sa_numeric
0,PlanSverige-10154245440447004,1,Hur är det att föda barn i ett land som drabba...,"[{'label': 'positive', 'score': 0.104590408504...",negative,0.787572,-1
1,PlanSverige-10154245440447004,2,På Plan International har vi löst det så att v...,"[{'label': 'positive', 'score': 0.420843392610...",negative,0.440226,-1
2,PlanSverige-10154245440447004,3,Hittills har 13 033 kvinnor fått hjälp och sup...,"[{'label': 'positive', 'score': 0.759050607681...",positive,0.759051,1
3,oxfamsverige-654404778032860,1,Lilla Jana 16 månader är en av miljontals männ...,"[{'label': 'positive', 'score': 0.264799565076...",negative,0.573133,-1
4,oxfamsverige-654404778032860,2,Familjen bor nu i ett flyktingläger i grannlan...,"[{'label': 'positive', 'score': 0.467231452465...",positive,0.467231,1


In [34]:
# Create a new column by multiplying the numeric label with the confidence score
sentences_df['sa_scaled'] = sentences_df['sa_numeric'] * sentences_df['sa_score']

sentences_df.head()


,id,sentence_number,sentence,sentiment,sa_label,sa_score,sa_numeric,sa_scaled
0,PlanSverige-10154245440447004,1,Hur är det att föda barn i ett land som drabba...,"[{'label': 'positive', 'score': 0.104590408504...",negative,0.787572,-1,-0.787572
1,PlanSverige-10154245440447004,2,På Plan International har vi löst det så att v...,"[{'label': 'positive', 'score': 0.420843392610...",negative,0.440226,-1,-0.440226
2,PlanSverige-10154245440447004,3,Hittills har 13 033 kvinnor fått hjälp och sup...,"[{'label': 'positive', 'score': 0.759050607681...",positive,0.759051,1,0.759051
3,oxfamsverige-654404778032860,1,Lilla Jana 16 månader är en av miljontals männ...,"[{'label': 'positive', 'score': 0.264799565076...",negative,0.573133,-1,-0.573133
4,oxfamsverige-654404778032860,2,Familjen bor nu i ett flyktingläger i grannlan...,"[{'label': 'positive', 'score': 0.467231452465...",positive,0.467231,1,0.467231


In [38]:
# Aggregating the data by 'id'
aggregated_df = sentences_df.groupby('id').agg(
    sa_numeric_mean=('sa_numeric', 'mean'),
    sa_numeric_sum=('sa_numeric', 'sum'),
    sa_scaled_mean=('sa_scaled', 'mean'),
    sa_scaled_sum=('sa_scaled', 'sum'),
    sentence_count=('sentence_number', 'max')
).reset_index()

aggregated_df.head()


,id,sa_numeric_mean,sa_numeric_sum,sa_scaled_mean,sa_scaled_sum,sentence_count
0,AMREFNordic-1152564321423108,1.000000,2,0.612928,1.225856,2
1,AMREFNordic-1152689281410612,0.200000,1,0.156740,0.783701,5
2,AMREFNordic-1155378134475060,-1.000000,-1,-0.491937,-0.491937,1
3,AMREFNordic-1155379041141636,1.000000,1,0.521695,0.521695,1
4,AMREFNordic-1155984904414383,0.333333,1,0.201763,0.605289,3


In [39]:
aggregated_df.tail()

,id,sa_numeric_mean,sa_numeric_sum,sa_scaled_mean,sa_scaled_sum,sentence_count
10419,weeffect-10153501350716533,-0.500000,-2,-0.322265,-1.289061,4
10420,weeffect-10153501462101533,0.000000,0,0.017261,0.034522,2
10421,weeffect-10153503169686533,-0.333333,-1,-0.235579,-0.706738,3
10422,weeffect-10153509707551533,0.333333,1,0.240769,0.722307,3
10423,weeffect-10153518540926533,-1.000000,-2,-0.487969,-0.975937,2


In [40]:
# Specify your desired output file path
fp = 'fb-sa-231228-001.tsv'
# Save or load the DataFrame as a TSV file
aggregated_df.to_csv(fp, sep='\t', index=False)
